In [2]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import mysql.connector
import networkx as nx
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon,MultiPolygon,LineString,MultiLineString,LinearRing,Point
from shapely import symmetric_difference
from shapely.ops import nearest_points
from shapely.wkt import loads
import scipy
import time
import numpy as np
import random
from networkx.drawing.nx_pydot import graphviz_layout
import matplotlib.pyplot as plt
import osmnx as ox
import glob
from itertools import combinations,product
from copy import deepcopy
import os
from lib_contour_lines import *
import mysql.connector


In [3]:

cnx = mysql.connector.connect(user='spirz', 
                              password='this_is_my_PASSWORD_m8',
                              host='localhost',
                              database='dem_from_contours_lines',
                              autocommit=True)
cursor = cnx.cursor(buffered=True,dictionary=True)
contours_lines_table_name='contours_lines'
tree_edges_table_name="tree_edges"
osm_nodes_table_name="osm_nodes"
osm_intersections_table_name="osm_intersections"
2

2

In [8]:
t1=time.time()
place_name='Corse, France'
G_osm=ox.graph_from_place(place_name,network_type='drive')

t2=time.time()
print(t2-t1)
len(G_osm.nodes()),len(G_osm.edges())

87.40194582939148


(17435, 37758)

In [6]:
t1=time.time()
cmd="SELECT * FROM %s"%osm_intersections_table_name
cursor.execute(cmd)
intersection=pd.DataFrame(cursor.fetchall())
t2=time.time()
print(t2-t1)

0.43584537506103516


In [19]:
intersection['edge']=intersection.apply(lambda row: (row['osm_begin'],row['osm_end'],row['osm_key']),axis=1)
edges=intersection['edge'].unique()
edges

array([(14468319, 15905397, 0), (14468319, 1434053161, 0),
       (14468319, 1434053206, 0), ..., (11291956061, 11291956068, 0),
       (11291956068, 11291961272, 0), (11329589123, 11329589133, 0)],
      dtype=object)

In [24]:
edge=random.choice(edges)
edge_df=intersection[intersection.edge==edge]
edge_df

,osm_begin,osm_end,osm_key,contour_line_id,edge_coordinate,elevation,edge
80567,6138194033,6138194037,0,113962,16.1543,360.0,"(6138194033, 6138194037, 0)"
80568,6138194033,6138194037,0,113960,38.1009,355.0,"(6138194033, 6138194037, 0)"
